This notebook illustrates geographic distribution of deployment of different electronic health record (EHR) software across England by CCG.

This is generated from a file available on request from NHS Digital.

In [1]:
##import libraries neededd
from ebmdatalab import bq
from ebmdatalab import maps
import pandas as pd


In [2]:
sql = """
WITH
  practice_count AS (
  SELECT
    ccg_id,
    COUNT(*) AS total_practices
  FROM
    hscic.practices
  WHERE
    setting = 4 ##this is the code for a "normal" general practice in NHS BSA terms
    AND status_code = 'A'
  GROUP BY
    ccg_id)
SELECT
  TRIM(Principal_Supplier) AS supplier,
  practices.ccg_id AS pct,
  COUNT(*) AS count,
  total_practices
FROM
  hscic.vendors software ##this is the BQ place where we have saved NHS Digital monthly extracts
JOIN
  hscic.practices
ON
  practices.code = software.ODS
JOIN
  practice_count
ON
  practice_count.ccg_id = practices.ccg_id
WHERE
  practices.setting = 4
  AND practices.status_code = 'A'
  AND Date = '2018-12-01' ###latest data is December
GROUP BY
  supplier,
  practices.ccg_id, total_practices
  
  """

ehr_by_ccg = bq.cached_read(sql, csv_path="ehr_by_ccg.csv")

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


In [3]:
##create an extra column with % deplyment across a CCG
ehr_by_ccg['percent'] = (ehr_by_ccg['count'] / ehr_by_ccg['total_practices'] * 100).round()
ehr_by_ccg.head(5)

,supplier,pct,count,total_practices,percent
0,TPP,00C,11,11,100.0
1,TPP,00D,31,36,86.0
2,EMIS,00D,4,36,11.0
3,TPP,00J,18,30,60.0
4,EMIS,00J,12,30,40.0


In [4]:
import importlib
importlib.reload(maps)
for supplier in ehr_by_ccg.supplier.unique():
    if not supplier:
        continue
    plt = maps.ccg_map(
        ehr_by_ccg[ehr_by_ccg['supplier'] == supplier], 
        title=supplier, column='percent', 
        separate_london=False)
    plt.show()

BaseException: Data contains CCG codes we don't know about: ['112']